In [27]:
import numpy as np
import pandas as pd

from datetime import date
import pandas_market_calendars as mcal

from statsmodels.tsa.stattools import coint

In [28]:
data_file=pd.read_csv('./data/stock_database.csv')
#data_file.head()

In [29]:
data_file['Date']=data_file['Date'].astype('datetime64[ns]')
data_file.set_index('Date', inplace=True)
data_file.drop(['ARNC', 'FOX', 'FOXA', 'PX', 'SCG', 'AKAM', 'CHK'], axis=1, inplace=True)

smoothing='yes'
data_file.to_csv('./data/selected.csv')


In [30]:
#start='1990-01-01'
#end='2020-11-06'
#nyse = mcal.get_calendar('NYSE')
#time_range=nyse.schedule(start_date=start, end_date=end)
#mcal.date_range(time_range, frequency='1W')

In [31]:
sp_list=pd.read_csv('./data/sp_list.csv')
sp_list=sp_list[['Symbol', 'Sector']].copy()

In [32]:
temp_list=[]
main_list=[]

stock_list=data_file.columns.to_list()
for i in range(0, len(stock_list)-1):
    for j in range(i+1, len(stock_list)):
        pair=data_file.iloc[:,[i,j]]
        pair.dropna(how='any', axis=0, inplace=True)
        
        if smoothing=='yes':
            pair=pair.rolling(window=5).mean()
            pair.dropna(inplace=True)
        
        try:
            result=coint(pair.iloc[:,0], pair.iloc[:,1])
        except ValueError:
            print(f'i:{i},j:{j}')
            
        temp_list=[result[0], result[1], result[2][0], result[2][1], result[2][2], i, j]
        main_list.append(temp_list)

test_results=pd.DataFrame(main_list, columns=('t_stat', 'p_value', 'CI_1', 'CI_5','CI_10', 'i', 'j'))

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110215 entries, 0 to 110214
Data columns (total 7 columns):
t_stat     110215 non-null float64
p_value    110215 non-null float64
CI_1       110215 non-null float64
CI_5       110215 non-null float64
CI_10      110215 non-null float64
i          110215 non-null int64
j          110215 non-null int64
dtypes: float64(5), int64(2)
memory usage: 5.9 MB


In [34]:
coint_results=test_results.loc[test_results.p_value<0.005]
coint_results=coint_results.sort_values(by='p_value')


In [35]:
new_cols=['i_sybl', 'j_sybl', 'i_sect', 'j_sect']
coint_results=coint_results.reindex(columns=coint_results.columns.tolist()+new_cols)

stock_list=data_file.columns.tolist()
for num in range(0, len(coint_results.index)):

    i=coint_results.iloc[num, 5]
    j=coint_results.iloc[num, 6]
    
    coint_results['i_sybl'].iloc[num]=stock_list[i]
    coint_results['j_sybl'].iloc[num]=stock_list[j]

coint_results=pd.merge(coint_results, sp_list, how='left', left_on='i_sybl', right_on='Symbol')
coint_results['i_sect']=coint_results['Sector']
coint_results.drop(columns=['Sector'], axis=1, inplace=True)
coint_results=pd.merge(coint_results, sp_list, how='left', left_on='j_sybl', right_on='Symbol')
coint_results['j_sect']=coint_results['Sector']
coint_results.drop(columns=['Symbol_x', 'Symbol_y', 'Sector'], axis=1, inplace=True)
coint_results.head()    

,t_stat,p_value,CI_1,CI_5,CI_10,i,j,i_sybl,j_sybl,i_sect,j_sect
0,-11.889913,6.815615e-21,-3.899889,-3.338053,-3.045785,19,433,AIG,V,Financials,Information Technology
1,-9.936845,3.581770e-16,-3.898590,-3.337329,-3.045282,187,224,GLW,ILMN,Information Technology,Health Care
2,-9.381288,9.238880e-15,-3.897850,-3.336917,-3.044996,286,447,MKC,WEC,Consumer Staples,Utilities
3,-9.249477,2.005234e-14,-3.899887,-3.338052,-3.045784,19,348,AIG,PM,Financials,Consumer Staples
4,-8.306248,5.099555e-12,-3.899716,-3.337957,-3.045718,155,407,ETFC,TEL,Financials,Information Technology


In [36]:
if smoothing=='yes':
    coint_results.to_csv('./data/results_smoothing.csv', index=False)
else:
    coint_results.to_csv('./data/results.csv', index=False)